In [1]:
!"C:/Users/alex/AppData/Local/Programs/Python/Python312/python.exe" -m pip install sacrebleu

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn as nn
import pandas as pd
import sentencepiece
from tqdm import tqdm
import comet_ml
import sacrebleu
from torch.utils.data import DataLoader

In [4]:
from dataset import WordTokenizer, TextDataset

In [5]:
tokenizer_en = WordTokenizer().fit("data\\train.de-en.en")
tokenizer_de = WordTokenizer().fit("data\\train.de-en.de")

In [6]:
dataset = TextDataset("data/train.de-en.de", "data/train.de-en.en")

In [7]:
from model import LanguageModel

In [8]:
loader = DataLoader(dataset, 64, True)

In [9]:
model = LanguageModel(dataset, embed_size=256, hidden_size=256, rnn_layers=6)

In [10]:
model.to("cuda")

LanguageModel(
  (embedding_en): Embedding(19065, 256, padding_idx=0)
  (embedding_de): Embedding(25408, 256, padding_idx=0)
  (rnn_encoder): RNN(256, 256, num_layers=6, batch_first=True)
  (rnn_decoder): RNN(256, 256, num_layers=6, batch_first=True)
  (linear): Linear(in_features=256, out_features=19065, bias=True)
)

In [11]:
for ind, lengths in loader:
    print(ind[1].shape)
    break

torch.Size([64, 128])


In [12]:
from train import train

In [13]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Всего параметров: {total_params:,}")

Всего параметров: 17,863,801


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), 1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 30)
   
train(model, optimizer, train_loader=loader, scheduler=scheduler, val_loader=loader, num_epochs=30 )

Training 1/30:  98%|█████████▊| 2999/3062 [01:47<00:02, 27.84it/s]